<a href="https://colab.research.google.com/github/ShreshthaSinha/Air-Quality-Prediction-/blob/main/Air_Quality_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project:** Air Quality Forecasting using Machine Learning


# **Goal:**  To Predict short-term and medium-term pollutant levels (e.g., PM2.5) using sensor and weather data, enabling proactive health & policy actions.

# **Objectives:**
- Build forecasting models (baseline → advanced ML/DL)
- Create a full pipeline: data ingestion → preprocessing → modeling → evaluation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#IMPORT LIBRARIES

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# READING DATASETS

station_hour = pd.read_excel('/content/drive/MyDrive/FSP PROJECT/station-hour.xlsx')
station = pd.read_excel('/content/drive/MyDrive/FSP PROJECT/station.xlsx')

In [6]:
station_hour.head()

,StationId,Date,Time,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,AP001,2017-11-25,09:00:00,104.00,148.50,1.93,23.00,13.75,9.80,0.1,15.30,117.62,0.30,10.40,0.23,155,Moderate
1,AP001,2017-11-25,10:00:00,94.50,142.00,1.33,16.25,9.75,9.65,0.1,17.00,136.23,0.28,7.10,0.15,159,Moderate
2,AP001,2017-11-25,11:00:00,82.75,126.50,1.47,14.83,9.07,9.70,0.1,15.40,149.92,0.20,4.55,0.08,173,Moderate
3,AP001,2017-11-25,14:00:00,68.50,117.00,1.35,13.60,8.35,7.40,0.1,21.80,161.70,0.10,2.30,0.00,191,Moderate
4,AP001,2017-11-25,15:00:00,69.25,112.25,1.52,11.80,7.55,9.25,0.1,21.38,161.68,0.10,2.35,0.00,191,Moderate


In [7]:
station.head()

,StationId,StationName,City,State,Status
0,AP001,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
1,AP005,"GVM Corporation, Visakhapatnam - APPCB",Visakhapatnam,Andhra Pradesh,Active
2,AS001,"Railway Colony, Guwahati - APCB",Guwahati,Assam,Active
3,BR005,"DRM Office Danapur, Patna - BSPCB",Patna,Bihar,Active
4,BR006,"Govt. High School Shikarpur, Patna - BSPCB",Patna,Bihar,Active


In [8]:
station_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219037 entries, 0 to 219036
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   StationId   219037 non-null  object        
 1   Date        219037 non-null  datetime64[ns]
 2   Time        219037 non-null  object        
 3   PM2.5       211535 non-null  float64       
 4   PM10        214232 non-null  float64       
 5   NO          219037 non-null  float64       
 6   NO2         217683 non-null  float64       
 7   NOx         217839 non-null  float64       
 8   NH3         219037 non-null  float64       
 9   CO          216699 non-null  float64       
 10  SO2         219037 non-null  float64       
 11  O3          219037 non-null  float64       
 12  Benzene     219037 non-null  float64       
 13  Toluene     218942 non-null  float64       
 14  Xylene      219037 non-null  float64       
 15  AQI         219037 non-null  int64         
 16  AQ

In [9]:
station_hour['AQI_Bucket'].value_counts()

,count
AQI_Bucket,
Moderate,93653
Satisfactory,75287
Good,25788
Poor,12547
Very Poor,9685
Severe,2077


In [10]:
station_hour.isnull().sum()

,0
StationId,0
Date,0
Time,0
PM2.5,7502
PM10,4805
NO,0
NO2,1354
NOx,1198
NH3,0
CO,2338


** Since the data is highly skewed, we will replace the null values with median values of the particular column.**

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
#REPLACING THE MISSING VALUES WITH THE MEDIAN VALUES OF THE RESOECTIVE COLUMNS

station_hour['PM2.5'].fillna(station_hour['PM2.5'].median(), inplace=True)
station_hour['PM10'].fillna(station_hour['PM10'].median(), inplace=True)

In [13]:
station_hour.isnull().sum()

,0
StationId,0
Date,0
Time,0
PM2.5,0
PM10,0
NO,0
NO2,1354
NOx,1198
NH3,0
CO,2338


In [14]:
#REPLACING THE MISSING VALUES WITH THE MEDIAN VALUES OF THE RESOECTIVE COLUMNS

station_hour['NO2'].fillna(station_hour['NO2'].median(), inplace=True)
station_hour['NOx'].fillna(station_hour['NOx'].median(), inplace=True)
station_hour['CO'].fillna(station_hour['CO'].median(), inplace=True)
station_hour['Toluene'].fillna(station_hour['Toluene'].median(), inplace=True)

In [15]:
station_hour.isnull().sum()

,0
StationId,0
Date,0
Time,0
PM2.5,0
PM10,0
NO,0
NO2,0
NOx,0
NH3,0
CO,0


**station_hour datasets has many zero values which are just null values, so we convert it to null values.**

In [16]:
station_hour.replace(0.0, np.nan, inplace=True)

In [17]:
station_hour[station_hour['NOx'] == 0]

,StationId,Date,Time,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket


In [18]:
station_hour[station_hour['CO'] == 0]

,StationId,Date,Time,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket


In [19]:
station_hour[station_hour==0]

,StationId,Date,Time,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219032,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219033,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219034,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219035,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
station_hour.isnull().sum()

,0
StationId,0
Date,0
Time,0
PM2.5,0
PM10,0
NO,0
NO2,0
NOx,981
NH3,0
CO,10794


**Replacing the changed null values to median values of the column.**

In [21]:
station_hour['Benzene'].fillna(station_hour['Benzene'].median(), inplace=True)
station_hour['NOx'].fillna(station_hour['NOx'].median(), inplace=True)
station_hour['CO'].fillna(station_hour['CO'].median(), inplace=True)
station_hour['Toluene'].fillna(station_hour['Toluene'].median(), inplace=True)

In [22]:
station_hour.isnull().sum()

,0
StationId,0
Date,0
Time,0
PM2.5,0
PM10,0
NO,0
NO2,0
NOx,0
NH3,0
CO,0


**Since XYLENE column has more than 25% null values, we are dropping it.**

In [23]:
station_hour.drop(columns=['Xylene'], inplace=True)

In [24]:
station_hour.columns

Index(['StationId', 'Date', 'Time', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3',
       'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'AQI', 'AQI_Bucket'],
      dtype='object')

**Changing the TIME column to DateTime type**

In [25]:
station_hour['Time'] = pd.to_datetime(station_hour['Time'], format='%H:%M:%S')

In [26]:
station_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219037 entries, 0 to 219036
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   StationId   219037 non-null  object        
 1   Date        219037 non-null  datetime64[ns]
 2   Time        219037 non-null  datetime64[ns]
 3   PM2.5       219037 non-null  float64       
 4   PM10        219037 non-null  float64       
 5   NO          219037 non-null  float64       
 6   NO2         219037 non-null  float64       
 7   NOx         219037 non-null  float64       
 8   NH3         219037 non-null  float64       
 9   CO          219037 non-null  float64       
 10  SO2         219037 non-null  float64       
 11  O3          219037 non-null  float64       
 12  Benzene     219037 non-null  float64       
 13  Toluene     219037 non-null  float64       
 14  AQI         219037 non-null  int64         
 15  AQI_Bucket  219037 non-null  object        
dtypes:

**Merging the two datasets based on StationId column.**

In [27]:
merged_df = pd.merge(station_hour, station, on='StationId', how='inner')  # Only matching IDs

In [28]:
merged_df

,StationId,Date,Time,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,AQI,AQI_Bucket,StationName,City,State,Status
0,AP001,2017-11-25,1900-01-01 09:00:00,104.00,148.50,1.93,23.00,13.75,9.80,0.10,15.30,117.62,0.30,10.40,155,Moderate,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
1,AP001,2017-11-25,1900-01-01 10:00:00,94.50,142.00,1.33,16.25,9.75,9.65,0.10,17.00,136.23,0.28,7.10,159,Moderate,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
2,AP001,2017-11-25,1900-01-01 11:00:00,82.75,126.50,1.47,14.83,9.07,9.70,0.10,15.40,149.92,0.20,4.55,173,Moderate,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
3,AP001,2017-11-25,1900-01-01 14:00:00,68.50,117.00,1.35,13.60,8.35,7.40,0.10,21.80,161.70,0.10,2.30,191,Moderate,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
4,AP001,2017-11-25,1900-01-01 15:00:00,69.25,112.25,1.52,11.80,7.55,9.25,0.10,21.38,161.68,0.10,2.35,191,Moderate,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219032,WB010,2020-02-14,1900-01-01 11:00:00,85.50,161.35,16.15,65.22,81.32,39.08,0.79,8.60,20.47,57.14,51.31,222,Poor,"Jadavpur, Kolkata - WBPCB",Kolkata,West Bengal,Active
219033,WB010,2020-02-14,1900-01-01 12:00:00,73.75,143.65,8.40,52.65,61.08,38.53,0.66,9.82,26.58,53.71,51.38,219,Poor,"Jadavpur, Kolkata - WBPCB",Kolkata,West Bengal,Active
219034,WB010,2020-02-14,1900-01-01 13:00:00,71.50,133.38,5.60,45.03,50.62,42.62,0.55,9.57,28.28,56.80,56.27,217,Poor,"Jadavpur, Kolkata - WBPCB",Kolkata,West Bengal,Active
219035,WB010,2020-02-14,1900-01-01 14:00:00,54.47,117.12,4.20,39.00,43.17,48.02,0.62,9.20,31.63,56.04,55.58,215,Poor,"Jadavpur, Kolkata - WBPCB",Kolkata,West Bengal,Active


In [29]:
merged_df.isnull().sum()

,0
StationId,0
Date,0
Time,0
PM2.5,0
PM10,0
NO,0
NO2,0
NOx,0
NH3,0
CO,0


**Feature Engineering**

In [32]:
def create_features(df, target_cols, date_col="Date", time_col="Time"):
    df = merged_df.copy()

    # Combine Date + Time
    df['datetime'] = df[date_col] + pd.to_timedelta(df[time_col].dt.strftime('%H:%M:%S'))
    df = df.sort_values('datetime')

    # Keep other pollutant columns as they are
    feature_cols = [c for c in df.columns if c not in [date_col, time_col, "datetime", "AQI", "AQI_Bucket", "StationId", "StationName", "City", "State", "Status"] + target_cols]

    # Add lags and rolling means for each target column
    important_lags = [24, 48, 72, 96]
    for target_col in target_cols:
        for lag in important_lags:
            df[f"{target_col}_lag{lag}"] = df[target_col].shift(lag)

        # Add rolling means for target
        for window in [24, 48]:
            df[f"{target_col}mean{window}h"] = df[target_col].rolling(window).mean()


    # Calendar features
    df['hour'] = df['datetime'].dt.hour
    df['dayofweek'] = df['datetime'].dt.dayofweek
    df['month'] = df['datetime'].dt.month

    # Cyclical encoding
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
    df['dow_sin'] = np.sin(2 * np.pi * df['dayofweek']/7)
    df['dow_cos'] = np.cos(2 * np.pi * df['dayofweek']/7)

    df = df.dropna().reset_index(drop=True)
    return df

pollutant_cols = ['PM2.5','PM10','NO','NO2','NOx','NH3','CO','SO2','O3','Benzene','Toluene']
df_features = create_features(merged_df, pollutant_cols)

In [34]:
df_features.columns

Index(['StationId', 'Date', 'Time', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3',
       'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'AQI', 'AQI_Bucket',
       'StationName', 'City', 'State', 'Status', 'datetime', 'PM2.5_lag24',
       'PM2.5_lag48', 'PM2.5_lag72', 'PM2.5_lag96', 'PM2.5mean24h',
       'PM2.5mean48h', 'PM10_lag24', 'PM10_lag48', 'PM10_lag72', 'PM10_lag96',
       'PM10mean24h', 'PM10mean48h', 'NO_lag24', 'NO_lag48', 'NO_lag72',
       'NO_lag96', 'NOmean24h', 'NOmean48h', 'NO2_lag24', 'NO2_lag48',
       'NO2_lag72', 'NO2_lag96', 'NO2mean24h', 'NO2mean48h', 'NOx_lag24',
       'NOx_lag48', 'NOx_lag72', 'NOx_lag96', 'NOxmean24h', 'NOxmean48h',
       'NH3_lag24', 'NH3_lag48', 'NH3_lag72', 'NH3_lag96', 'NH3mean24h',
       'NH3mean48h', 'CO_lag24', 'CO_lag48', 'CO_lag72', 'CO_lag96',
       'COmean24h', 'COmean48h', 'SO2_lag24', 'SO2_lag48', 'SO2_lag72',
       'SO2_lag96', 'SO2mean24h', 'SO2mean48h', 'O3_lag24', 'O3_lag48',
       'O3_lag72', 'O3_lag96', 'O3mean24h', 'O3mea

In [35]:
print("Columns in df_features:", len(df_features.columns))
print(df_features.columns.tolist())

Columns in df_features: 94
['StationId', 'Date', 'Time', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'AQI', 'AQI_Bucket', 'StationName', 'City', 'State', 'Status', 'datetime', 'PM2.5_lag24', 'PM2.5_lag48', 'PM2.5_lag72', 'PM2.5_lag96', 'PM2.5mean24h', 'PM2.5mean48h', 'PM10_lag24', 'PM10_lag48', 'PM10_lag72', 'PM10_lag96', 'PM10mean24h', 'PM10mean48h', 'NO_lag24', 'NO_lag48', 'NO_lag72', 'NO_lag96', 'NOmean24h', 'NOmean48h', 'NO2_lag24', 'NO2_lag48', 'NO2_lag72', 'NO2_lag96', 'NO2mean24h', 'NO2mean48h', 'NOx_lag24', 'NOx_lag48', 'NOx_lag72', 'NOx_lag96', 'NOxmean24h', 'NOxmean48h', 'NH3_lag24', 'NH3_lag48', 'NH3_lag72', 'NH3_lag96', 'NH3mean24h', 'NH3mean48h', 'CO_lag24', 'CO_lag48', 'CO_lag72', 'CO_lag96', 'COmean24h', 'COmean48h', 'SO2_lag24', 'SO2_lag48', 'SO2_lag72', 'SO2_lag96', 'SO2mean24h', 'SO2mean48h', 'O3_lag24', 'O3_lag48', 'O3_lag72', 'O3_lag96', 'O3mean24h', 'O3mean48h', 'Benzene_lag24', 'Benzene_lag48', 'Benzene_lag72', 'Benzene_lag

In [38]:
merged_df.columns

Index(['StationId', 'Date', 'Time', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3',
       'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'AQI', 'AQI_Bucket',
       'StationName', 'City', 'State', 'Status'],
      dtype='object')